# Basic Model Setup
I am skipping the exploration of tokenization methods etc, as I tried to complete this in org mode but ultimately crashed my session trying to run the model on my laptop. For ease of use, I'm replicating this in a notebook and will run it in colab.

In [1]:
from fastai.text.all import *
path=untar_data(URLs.IMDB)

In [3]:
get_imdb = partial(get_text_files, folders = ['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path,path=path,bs=128,seq_len=80)

In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM,drop_mult=0.3,
                              metrics=[accuracy,Perplexity()]).to_fp16

This model takes a very long time to run -- so it is good to save our model state periodically. I wonder how I download that from colab. Will have to see.

In [ ]:
learn.save('1epoch')

In [ ]:
learn=learn.load('1epoch')
learn.unfreeze()
learn.fit_one_cycle(10,2e-3)

Once we complete this, we can save the whole model *except for the final layer that converts activations to probabilities of picking each token*. The model without the final layer is called the *encoder*. We can use this as the foundation for, in this example, a sentiment analyzer.